In [11]:
import psycopg2
import geopandas as gpd
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from config import config

# CONEXIÓN a la BASE DE DATOS

params = config()
conn = psycopg2.connect(**params)

comarca = 'jerte'
zona = 'jerte'

# CONDICIONES SINÓPTICAS

sql = '''WITH inc as 
        (SELECT a.*, b.descripcion_corta as causa
        FROM 
            incendios_proximos_ze a,
            causas b
        WHERE 
            a.idcausa = b.id
            and b.ididioma = 0
        ORDER by tot desc)
        SELECT inc.*, c.texto, c.descripcion
        FROM 
            inc,
            texto_situacion_sinoptica c
        WHERE
            c.idpif = inc.idpif
        '''

incendios_cs = gpd.read_postgis(sql, conn)
incendios_cs['epa'] = (pd.DatetimeIndex(incendios_cs.deteccion).month > 5) & (pd.DatetimeIndex(incendios_cs.deteccion).month < 11)
ss = incendios_cs[['idpif', 'descripcion']].groupby(['descripcion']).count()
ss_epa = incendios_cs[incendios_cs.epa == True][['idpif', 'descripcion']].groupby(['descripcion']).count()
ss_epb = incendios_cs[incendios_cs.epa == False][['idpif', 'descripcion']].groupby(['descripcion']).count()
ss = ss.merge(right=ss_epa, left_index=True, right_index=True, how='left')
ss = ss.merge(right=ss_epb, left_index=True, right_index=True, how='left')
ss.columns = ['Total', 'EPA', 'EPB']
ss = ss[['EPA', 'EPB', 'Total']]
ss = ss.fillna(0)
ss = ss.convert_dtypes()
ss.index.name = 'Situación sinóptica'

# CAUSAS

causas = pd.read_sql('''
            with c as 
                (select idcausa / 100 as ppal, count(*) as num, 
                sum(superficiearboladatotal) as sup_arbolada, 
                sum(superficinoarboladatotal) as sup_no_arbolada 
                from incendios_proximos_ze 
                group by idcausa 
                order by idcausa) 
            select 
                ppal, 
                sum(num) as num_incendios, 
                sum(sup_arbolada) as sup_arbolada, 
                sum(sup_no_arbolada) as sup_no_arbolada,
                sum(sup_arbolada) + sum(sup_no_arbolada) as sup_total
            from 
                c
            group by ppal 
            order by ppal
            ''', conn)

causas_detalle = pd.read_sql('''
            with c as 
                (select idcausa, count(*) as num, 
                sum(superficiearboladatotal) as sup_arbolada, 
                sum(superficinoarboladatotal) as sup_no_arbolada 
                from incendios_proximos_ze 
                group by idcausa 
                order by idcausa) 
            select 
                idcausa,
                descripcion_corta,
                sum(num) as num_incendios, 
                sum(sup_arbolada) as sup_arbolada, 
                sum(sup_no_arbolada) as sup_no_arbolada,
                sum(sup_arbolada) + sum(sup_no_arbolada) as sup_total
            from 
                c,
                causas 
            where 
                c.idcausa = causas.id
                and causas.ididioma = 0
            group by 
                descripcion_corta,
                idcausa
            order by c.idcausa
            ''', conn)
    
causas.index = ['Rayo', 'Negligencia', 'Accidente', 'Intencionado', 'Desconocida', 'Reproducción']
causas = causas[['num_incendios', 'sup_arbolada', 'sup_no_arbolada', 'sup_total']]

# SEVERIDAD 

incendios = [{'zona': 'Jerte-Tornavacas', 'idpif': ['1133522', '1143015', '1143273', '574176', '1164099']}, 
             {'zona': 'Villanueva-Madrigal', 'idpif': ['1122656', '1132420', '1133464', '1163320', '1163391']}, 
             {'zona': 'Losar', 'idpif': ['1163407', '569396', '1169767', '574250', '1122672']}]

severidad = []

zonas_dict = {'jerte': 0, 'tornavacas': 0, 'madrigal': 1, 'losar': 2}

severidad = incendios[zonas_dict[zona]]
severidad['incendios'] = []

for n in severidad['idpif']:
    perimetro = gpd.read_postgis("select * from perimetros_utm where idpif = {}".format(n), conn)
    severidad['incendios'].append({'fecha': perimetro.iloc[0]['fecha'].strftime('%d/%m/%Y'), 'idpif': n})

# CATASTRO 

if zona == 'madrigal':
    sql = '''
        SELECT nombre, masa, st_union(geom) as geom, sum (area) / 10000 as sup
        FROM catastro_mod_{}
        GROUP by nombre, masa
        '''.format(zona)
    sql += ''' union all
            select nombre, masa, st_union(geom) as geom, sum(area) / 10000 as sup
            from catastro_mod_villanueva
            GROUP by nombre, masa
            '''
    sql += ''' union all
            select nombre, masa, st_union(geom) as geom, sum(area) / 10000 as sup
            from catastro_mod_valverde
            GROUP by nombre, masa
            '''
elif zona == 'losar':
    sql = 'select nombre, st_union(geom) as geom, sum(area) / 10000 as sup from catastro_mod_{} group by nombre'. format(zona)
else:
    sql = 'select nombre, masa, st_union(geom) as geom, sum(area) / 10000 as sup from catastro_mod_{} group by nombre, masa'. format(zona)

parcelas = gpd.read_postgis(sql, conn)
catastro = parcelas.to_dict(orient='records')

# FORESTALES
    
forestales = pd.read_sql("select forestal, contacto from forestales where zona = '{}'".format(zona), conn)

# HIDROGRAFÍA

rios = gpd.read_postgis('''select 
                row_number() over (order by a.long_km desc) as id_mapa, a.* 
                from rios a, zona_estudio_{} b 
                where st_intersects(a.geom, b.geom)
                and a.long_km < 50'''.format(zona), conn)

# METEO (TODO crear tabla en base de datos para no tener tanto código aquí)
datos_meteo = []
if comarca == 'vera':
    estaciones = ['madrigal', 'piornal']
else:
    estaciones = ['tornavacas', 'piornal']

for estacion in estaciones:
    dias_lluvia = pd.read_sql('''
                            select estacion, count(*) as dias_lluvia
                            from meteo_{} 
                            where prectotal > 0 and estacion is not null
                            and estacion != 'invierno_2020_2021'
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    dias_sin_lluvia_5 = pd.read_sql('''
                            select estacion, count(*) as dsl5
                            from meteo_{} 
                            where dias_sin_lluvia_5 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    dias_sin_lluvia_8 = pd.read_sql('''
                            select estacion, count(*) as dsl8
                            from meteo_{} 
                            where dias_sin_lluvia_8 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    nieve = pd.read_sql('''
                            select estacion, count(*) as nieve
                            from meteo_{}
                            where prectotal > 0 and tmin < 0 and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    heladas = pd.read_sql('''
                            select estacion, count(*) as heladas
                            from meteo_{}
                            where tmin < 0 and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    ventana_solana = pd.read_sql('''
                            select estacion, count(*) as vs
                            from meteo_{} 
                            where tmax < 21 and tmin > 6 and dias_sin_lluvia_5 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    ventana_umbria = pd.read_sql('''
                            select estacion, count(*) as vu
                            from meteo_{}
                            where tmax < 21 and tmin > 6 and dias_sin_lluvia_8 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)

    meteo = pd.merge(dias_lluvia, dias_sin_lluvia_5, how='left')
    meteo = pd.merge(meteo, dias_sin_lluvia_8, how='left')
    meteo = pd.merge(meteo, nieve, how='left')
    meteo = pd.merge(meteo, heladas, how='left')
    meteo = pd.merge(meteo, ventana_solana, how='left')
    meteo = pd.merge(meteo, ventana_umbria, how='left')
    meteo = meteo.set_index('estacion')
    meteo = meteo.fillna(0)
    meteo = meteo.convert_dtypes()
    meteo.loc['media'] = meteo.mean(axis=0)
    meteo.index.name = 'Periodo'
    meteo.columns=['días lluvia', '5 días sin lluvia', '8 días sin lluvia', 'nieve', 'heladas', 'ventana solana', 'ventana umbría']
    datos_meteo.append({'estacion': estacion, 'meteo':meteo})

# VEGETACIÓN

habitats_raw = pd.read_sql('''
                        select 
                            distinct codue, nom_comun, generico, nom_habita, 
                            alianza, spsalianza, 
                            prioritari, st_area(st_intersection(st_transform(a.geom, 25830), b.geom)) /10000 as sup 
                        from atlashabitats2005_extremadura a,
                        zona_estudio_{} b
                        where st_intersects(st_transform(a.geom, 25830), b.geom)
                        '''.format(zona), conn)

habitats_prioritarios = habitats_raw[habitats_raw.prioritari == '*'].groupby(['codue', 'nom_comun', 'generico', 'nom_habita', 'alianza', 'spsalianza']).sum()
habitats = habitats_raw[habitats_raw.prioritari == 'Np'][['codue', 'generico', 'sup']].groupby(['codue', 'generico']).sum().reset_index()
display(habitats.loc[habitats['codue'] == '4030'])
habitats_anexo = pd.read_sql('''
                        select 
                            distinct codue, nom_comun, generico, nom_habita, 
                            alianza, spsalianza
                        from atlashabitats2005_extremadura a,
                        zona_estudio_{} b
                        where st_intersects(st_transform(a.geom, 25830), b.geom)
                        and prioritari = 'Np'
                        '''.format(zona), conn).to_dict(orient='records')

# ÁRBOLES SINGULARES

arboles_singulares = gpd.read_postgis('''
                    select a.* from arboles_singulares a,
                    zona_estudio_{} b
                    where st_intersects(st_transform(a.geom, 25830), b.geom)
                    '''.format(zona), conn).to_dict(orient='records')

#FAUNA

cod = ['1308', '1078', '1088', '6149', '1044', '1194', '1065', '1301', '1046', '5371', '1259', '1083', '1355', '1036', '1221', '1338', '1310', '1323', '1307', '1321',  '1324', '1041', '1305', '1304', '1303', '1302', '1123']
nom_cient = ['Barbastella barbastellus', 'Callimorpha quadripunctaria', 'Cerambyx cerdo', 'Chondrostoma polylepis', 'Coenagrion mercuriale', 'Discoglossus galganoi', 'Euphydryas aurinia', 'Galemys pyrenaicus', 'Gomphus graslinii', 'Iberolacerta monticola', 'Lacerta schreiberi', 'Lucanus cervus', 'Lutra lutra', 'Macromia splendens', 'Mauremys leprosa', 'Microtus cabrerae', 'Miniopterus schreibersii', 'Myotis bechsteinii', 'Myotis blythii', 'Myotis emarginatus', 'Myotis myotis', 'Oxygastra curtisii', 'Rhinolophus euryale', 'Rhinolophus ferrumequinum', 'Rhinolophus hipposideros', 'Rhinolophus mehelyi', 'Rutilus alburnoides']    
nom_comun = ['murciélago de bosque', '-', 'longicornio de la encina', 'boga del Tajo', 'caballito del diablo', 'sapillo pintojo ibérico', 'doncella de ondas rojas', 'desmán ibérico', '-', 'lagartija carpetana', 'lagarto verdinegro', 'ciervo volante', 'nutria', '-', 'galapago leproso', 'topillo de la cabrera', 'murciélago de cueva', 'murciélago ratonero forestal', 'murciélago ratonero mediano', 'murciélago ratonero pardo', 'murciélago ratonero grande', '-',  'murciélago mediterraneo herradura', 'murciélago grande herradura', 'murciélago pequeño de herradura', 'murciélago mediano de herradura', 'calandino']
grupo = ['mamíferos quirópteros', 'invertebrados insectos', 'invertebrados insectos', 'peces', 'invertebrados insectos', 'anfibios', 'invertebrados insectos', 'mamíferos insectivoros', 'invertebrados insectos', 'reptiles', 'reptiles', 'invertebrados insectos', 'mamíferos carnívoros', 'invertebrados insectos', 'reptiles', 'mamíferos roedores', 'mamíferos quiropteros', 'mamíferos quiropteros', 'mamíferos quiropteros', 'mamíferos quiropteros', 'mamíferos quiropteros', 'invertebrados insectos', 'mamíferos quiropteros', 'mamíferos quiropteros', 'mamíferos quiropteros', 'mamíferos quiropteros', 'peces']
fauna = pd.DataFrame({'cod': cod, 'nombre científico': nom_cient, 'nombre común': nom_comun, 'grupo': grupo})

##########################
# PLAN DE QUEMAS !!!! ####
##########################

# SUPERFICIE

coordenadas = pd.read_sql('''
            SELECT a.nombre, 
                ST_X(ST_Centroid(ST_Transform(a.geom, 25830))) as coordenada_x, 
                ST_Y(ST_Centroid(ST_Transform(a.geom, 25830))) as coordenada_y,
                b.nombre
            FROM quema_{}_todas a, catastro_mod_{} b
            WHERE ST_Intersects(ST_Transform(a.geom, 25830), b.geom)
            ORDER BY a.parcela
                '''.format(zona, zona), conn)
coordenadas.columns = ['Parcela', 'Coordenada X', 'Coordenada Y', 'Finca']
display(coordenadas.drop_duplicates())

sup_df = pd.read_sql('''
            SELECT
                nombre, ST_Area(ST_Transform(geom, 25830)) / 10000 as sup
            FROM quema_{}_todas
            ORDER by parcela 
            '''.format(zona), conn)
sup_df.columns = ['Parcela', 'Superficie (ha)']

# #PENDIENTES Y ALTITUD

stats = pd.read_sql('''
            SELECT a.nombre,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(a.geom, 25830))))).min as altitud_min,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(a.geom, 25830))))).max as altitud_max,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(a.geom, 25830))))).mean as altitud_media,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(a.geom, 25830))))).mean as pendiente
            from quema_{}_todas a, pendientes25 b, mdt25 c
            GROUP BY a.nombre, a.parcela
            ORDER BY a.parcela
            '''.format(zona), conn)

stats.columns = ['Parcela', 'Altitud mínima', 'Altitud máxima', 'Altitud media', 'Pendiente']

# # ORIENTACIONES

orientaciones_df = pd.read_sql('''
            SELECT a.nombre,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(a.geom, 25830))))).mean as orientacion
            FROM quema_{}_todas a, orientaciones25 b
            GROUP BY a.nombre, a.parcela 
            ORDER BY a.parcela
            '''.format(zona), conn)    

orientaciones_df.columns = ['Parcela', 'Orientación']

# # HABITATS

habitats_parcelas = pd.read_sql('''
            WITH ab as (
                SELECT
                    a.nombre,
                    codue as codigo,
                    nom_comun, 
                    st_area(st_transform(a.geom, 25830)) / 10000 as sup_parcela,
                    st_area(st_intersection(b.geom, st_transform(a.geom, 23030))) / 10000 as superficie,
                    st_area(st_intersection(b.geom, st_transform(a.geom, 23030))) / st_area(st_transform(a.geom, 23030)) * 100 as porcentaje
                FROM atlashabitats2005_extremadura b, quema_{}_todas a
                WHERE st_intersects(st_transform(a.geom, 23030), b.geom)
                GROUP BY a.nombre, b.codue, b.nom_comun, a.parcela, b.geom, a.geom
                ORDER BY a.parcela
            )
            SELECT 
                ab.*,
                Sum(St_Area(St_Intersection(atlas.geom, St_Transform(zec.geom, 23030)))) / 10000 as sup_hab_zec,
                ab.superficie * 100 * 10000 / Sum(St_Area(St_Intersection(atlas.geom, St_Transform(zec.geom, 23030)))) as sup_hab_zec
            FROM ab, atlashabitats2005_extremadura atlas, sierra_gredos_y_valle_jerte zec
            WHERE st_intersects(atlas.geom, st_transform(zec.geom, 23030)) and ab.codigo = atlas.codue and ab.nom_comun = atlas.nom_comun
            GROUP BY ab.nombre, ab.codigo, ab.nom_comun, ab.sup_parcela, ab.superficie, ab.porcentaje
            '''.format(zona), conn)

habitats_parcelas.columns = ['Parcela', 'Código UE', 'Descripción', 'Superficie Parcela', 'Superficie del hábitat en la parcela', 'Porcentaje de parcela ocupado por el hábitat', 'Superficie del hábitat en el ZEC', 'Porcentaje de la superficie del hábitat en la parcela sobre el total del hábitat en el ZEC']
display(habitats_parcelas)
    
prescripcion = pd.DataFrame({
    'Factores meteorológicos': ['Temperatura', 'Humedad Relativa', 'Viento', 'Humedad del suelo', 'HCFM', 'Nº días sin llover'],
    'Rango base': ['5-20 ºC (heladas previas)', '30-60%', '5-15 km/h', '50-100%', '5-11%', '5-25 días'],
    'Rango práctico': ['6-21 ºC', '27 - 64 %%', '0-25 km/h', 'Húmedo', '6-10%', '2-12 días']
})

# FICHAS

parcelas = pd.read_sql('''
            SELECT
                a.nombre,
                a.parcela,
                ST_Area(ST_Transform(a.geom, 25830)) / 10000 as sup, 
                ST_Perimeter(ST_Transform(a.geom, 25830)) as perim,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).min as pend_min,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).max as pend_max,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).mean as pend_mean,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).min as alt_min,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).max as alt_max,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).mean as alt_mean,
                (ST_SummaryStats(St_Union(ST_Clip(d.rast, st_transform(geom, 25830))))).mean as orient
            FROM 
                quema_{}_todas a,
                pendientes25 b,
                mdt25 c,
                orientaciones25 d
            GROUP BY
                a.nombre, a.parcela, a.geom
            ORDER BY a.parcela
            '''.format(zona), conn)

display(parcelas)
# CERRAR CONEXIÓN BASE DE DATOS

conn.close()

# EXPORTAR 

capitulos = ['introduccion', 'objetivos', 'ambito', 'requisitos', 'puestos', 'planificacion', 'elementos', 'juicio']

env = Environment(loader=FileSystemLoader('static/templates_vera'))
template =env.get_template('base.html')
output = template.render(
    comarca = comarca,
    zona = zona,
    severidad = severidad,
    condiciones_sinopticas = incendios_cs.to_dict(orient='records'),
    ss = ss,
    catastro = catastro,
    forestales = forestales,
    capitulos = capitulos,
    rios = rios,
    datos_meteo = datos_meteo,
    causas = causas,
    causas_detalle = causas_detalle,
    habitats = habitats,
    habitats_prioritarios = habitats_prioritarios,
    habitats_anexo = habitats_anexo,
    arboles_singulares = arboles_singulares, 
    fauna = fauna,
    coordenadas = coordenadas,
    sup = sup_df,
    stats = stats,
    orientaciones = orientaciones_df,
    prescripcion = prescripcion,
    habitats_plan_quemas = habitats_parcelas,
    parcelas = parcelas.to_dict(orient='records')
)

with open('static/html/final.html', 'w') as f:
    f.write(output)

,codue,generico,sup
0,4030,Brezal-jarales,5.843333


,Parcela,Coordenada X,Coordenada Y,Finca
0,Parcela 1,266730.108666,4.460535e+06,Finca la Nijarra
1,Parcela 2,266660.314364,4.460678e+06,Finca la Nijarra
2,Parcela 3,266558.569654,4.460860e+06,Finca la Nijarra
3,Parcela 4,266632.129102,4.461065e+06,Finca la Nijarra
4,Parcela 5,266439.847593,4.461237e+06,Finca la Nijarra
5,Parcela 6,266250.134209,4.461306e+06,Finca la Nijarra
6,Parcela 7,265826.094956,4.461307e+06,Finca la Nijarra


,Parcela,Código UE,Descripción,Superficie Parcela,Superficie del hábitat en la parcela,Porcentaje de parcela ocupado por el hábitat,Superficie del hábitat en el ZEC,Porcentaje de la superficie del hábitat en la parcela sobre el total del hábitat en el ZEC
0,Parcela 1,5120,Piornales bejarano-gredenses de piorno serrano,0.968395,0.968417,100.000000,18104.673897,0.005349
1,Parcela 2,5120,Piornales bejarano-gredenses de piorno serrano,0.985962,0.985984,100.000000,18104.673897,0.005446
2,Parcela 3,5120,Piornales bejarano-gredenses de piorno serrano,1.185305,1.185332,100.000000,18104.673897,0.006547
3,Parcela 4,5120,Piornales bejarano-gredenses de piorno serrano,2.308032,1.025698,44.439369,18104.673897,0.005665
4,Parcela 4,5120,Piornales serranos orosubmediterráneos bejaran...,2.308032,1.272777,55.144310,8169.987064,0.015579
5,Parcela 4,6230,Cervunal carpetano-leonés occidental y laciano...,2.308032,0.009609,0.416320,1658.888946,0.000579
6,Parcela 5,6230,Cervunal carpetano-leonés occidental y laciano...,1.344451,1.344481,100.000000,1658.888946,0.081047
7,Parcela 6,6230,Cervunal carpetano-leonés occidental y laciano...,0.235113,0.235118,100.000000,1658.888946,0.014173
8,Parcela 7,5120,Piornales bejarano-gredenses de piorno serrano,9.008217,8.135834,90.313647,18104.673897,0.044938
9,Parcela 7,5120,Piornales serranos orosubmediterráneos bejaran...,9.008217,0.872587,9.686353,8169.987064,0.010680


,nombre,parcela,sup,perim,pend_min,pend_max,pend_mean,alt_min,alt_max,alt_mean,orient
0,Parcela 1,1,0.968395,442.810483,39.325924,56.133358,46.799985,1648.974976,1690.891968,1667.774919,216.341291
1,Parcela 2,2,0.985962,450.795967,29.564049,55.227169,39.947699,1673.677979,1702.722046,1690.550232,231.748589
2,Parcela 3,3,1.185305,608.747713,27.981104,44.838184,35.050270,1688.687012,1708.946045,1699.965663,238.432298
3,Parcela 4,4,2.308032,737.950302,9.574162,27.337339,18.509648,1706.413940,1742.499023,1727.183105,297.417070
4,Parcela 5,5,1.344451,519.039670,18.659676,44.328526,31.164894,1667.852051,1713.178955,1692.817424,257.310078
5,Parcela 6,6,0.235113,208.057582,28.944324,40.269112,33.859407,1657.525024,1664.010010,1659.718018,224.706512
6,Parcela 7,7,9.008217,1280.630191,22.722860,86.861816,48.276423,1546.432007,1737.223022,1649.737077,162.811174
